# FEATURE EXTRACTION
This notebook handles the feature extraction from our selected dataset, UrbanSounds8k, using librosa.
our choice of features is the following:
- MFCC
- Spectral Centroid
- Spectral Bandwidth
- Spectral Contrast
- Spectral Rolloff

install prerequisites: 
notebook breaks without resampy.
A couple of scripts included in the repo are intended to batch resample the dataset to 16khz sampling frequency.
Recently we found out that you can resample stuff directly in librosa, hence the need for resampy.

In [ ]:
#!pip install librosa
#!pip install resampy

In [ ]:

import os
import pandas as pd
import numpy as np
import librosa
from tqdm import tqdm

DATASET_PATH = "/home/george/MScAI/MachineLEarning/archive" 
OUTPUT_CSV = "urban_sound_features.csv"

METADATA_PATH = os.path.join(DATASET_PATH, "UrbanSound8K.csv")
AUDIO_PATH = os.path.join(DATASET_PATH, "")

In [ ]:
def extract_features(file_path):
    try:
        audio, sample_rate = librosa.load(file_path, sr=16000, res_type='kaiser_fast')
        
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
        mfccs_mean = np.mean(mfccs.T, axis=0)
        mfccs_std = np.std(mfccs.T, axis=0)
        
        cent = librosa.feature.spectral_centroid(y=audio, sr=sample_rate)
        cent_mean = np.mean(cent)
        cent_std = np.std(cent)
        
        bw = librosa.feature.spectral_bandwidth(y=audio, sr=sample_rate)
        bw_mean = np.mean(bw)
        bw_std = np.std(bw)
        
        contrast = librosa.feature.spectral_contrast(y=audio, sr=sample_rate)
        contrast_mean = np.mean(contrast.T, axis=0)
        contrast_std = np.std(contrast.T, axis=0)
        
        rolloff = librosa.feature.spectral_rolloff(y=audio, sr=sample_rate)
        rolloff_mean = np.mean(rolloff)
        rolloff_std = np.std(rolloff)
        
        return np.hstack([
            mfccs_mean, mfccs_std, 
            cent_mean, cent_std, 
            bw_mean, bw_std,
            contrast_mean, contrast_std,
            rolloff_mean, rolloff_std
        ])
        
    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return None

In [ ]:
if not os.path.exists(METADATA_PATH):
    print(f" Error: Metadata file not found at {METADATA_PATH}")
else:
    metadata = pd.read_csv(METADATA_PATH)
    print(f" Metadata loaded. Processing {metadata.shape[0]} files at 16kHz...")

    feature_list = []
    label_list = []

    for index, row in tqdm(metadata.iterrows(), total=metadata.shape[0]):
        
        fold_name = f"fold{row['fold']}"
        file_name = row["slice_file_name"]
        full_path = os.path.join(AUDIO_PATH, fold_name, file_name)
        
        data = extract_features(full_path)
        
        if data is not None:
            feature_list.append(data)
            label_list.append(row["class"])

In [ ]:
    df_features = pd.DataFrame(feature_list)
    df_features['class_label'] = label_list

    print(f"\n Extraction complete!")
    print(f"Data shape: {df_features.shape}")

    df_features.to_csv(OUTPUT_CSV, index=False)
    print(f" Data saved to: {os.path.abspath(OUTPUT_CSV)}")